In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Input,models,layers,optimizers,metrics
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.applications import EfficientNetB7
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

cp = ModelCheckpoint(monitor = 'val_loss',filepath = "C:/Users/Administrator/Downloads/mnist/models/Effi7.h5",save_best_only=True)
cp_ = ModelCheckpoint(monitor = 'val_accuracy',filepath = "C:/Users/Administrator/Downloads/mnist/models/Effi7_acc.h5",save_best_only=True)
es = EarlyStopping(monitor = 'val_loss',patience=15)
lr = ReduceLROnPlateau(monitor = 'val_loss',patience=6)
optimizer = Adam(lr=0.02)


train_datagen = ImageDataGenerator(rescale=1./255,horizontal_flip = True,width_shift_range=0.1,height_shift_range=0.1,fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale=1./255,horizontal_flip = True,width_shift_range=0.1,height_shift_range=0.1,fill_mode='nearest')
# train_generator = train_datagen.flow_from_directory('C:/Users/Administrator/Downloads/mnist/dirty_mnist_2nd/',target_size=(150,150),batch_size=4,class_mode='binary')
test_datagen = ImageDataGenerator(rescale=1./255,horizontal_flip = True,width_shift_range=0.1,height_shift_range=0.1,fill_mode='nearest')
# test_generator = test_datagen.flow_from_directory('C:/Users/Administrator/Downloads/mnist/test_dirty_mnist_2nd/',target_size=(150,150),batch_size=4,class_mode='binary')

transfer_model = EfficientNetB7(weights='imagenet',include_top=False,input_shape=(256,256,3))
transfer_model.trainable=False

model = models.Sequential()
model.add(transfer_model)
model.add(Flatten())
model.add(Dense(52,activation='relu'))
model.add(Dense(26,activation='sigmoid'))

In [ ]:
import numpy as np
import cv2

In [ ]:
def read_data(train=True):
    result = []
    if train:
        for i in range(50000):
            imgpath = "C:/Users/Administrator/Downloads/mnist/dirty_mnist_2nd/{}.png".format(str((5-len(str(i)))*'0')+str(i))
            img = cv2.imread(imgpath,cv2.IMREAD_GRAYSCALE)
            img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            result.append(img)
        return np.array(result).reshape(-1,256,256,3)/255.0
    else:
        for i in range(5000):
            imgpath="C:/Users/Administrator/Downloads/mnist/test_dirty_mnist_2nd/{}.png".format(str(50000+i))
            img = cv2.imread(imgpath,cv2.IMREAD_GRAYSCALE)
            img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            result.append(img)
        return np.array(result).reshape(-1,256,256,3)/255.0
    
x = read_data()
np.save('C:/Users/Administrator/Downloads/mnist/npy_x_all.npy',arr=x)
df = pd.read_csv("C:/Users/Administrator/Downloads/mnist/dirty_mnist_2nd_answer.csv",index_col=0)
y = df.values
x_train,x_val = train_test_split(x,test_size=0.15)
x_train,x_val,y_train,y_val = train_test_split(x,y,test_size=0.15)

In [ ]:
np.save('C:/Users/Administrator/Downloads/mnist/npy_x_all.npy',arr=x)
# np.save('C:/Users/Administrator/Downloads/mnist/npy_y_all.npy',arr=y)

In [ ]:
np.save('C:/Users/Administrator/Downloads/mnist/npy_x_train.npy',arr=x_train)
# np.save('C:/Users/Administrator/Downloads/mnist/npy_y_train.npy',arr=y_train)
np.save('C:/Users/Administrator/Downloads/mnist/npy_x_val.npy',arr=x_val)
# np.save('C:/Users/Administrator/Downloads/mnist/npy_y_val.npy',arr=y_val)

In [ ]:
x.shape,y.shape

In [ ]:
train_generator = train_datagen.flow(x_train,y_train,batch_size=16)
val_generator = val_datagen.flow(x_val,y_val)

model.compile(loss = 'binary_crossentropy',optimizer = Adam,metrics=['accuracy'])
model.fit_generator(train_generator,validation_data = val_generator,epochs=100,callbacks = [es,lr,cp,cp_])